# **Infer Soma And Neurites From Cellmask**

***Prior to this notebook, you should have already run through [1.0_image_setup](1.0_image_setup.ipynb) and [1.1_infer_masks](1.1_infer_masks_from-composite_with_nuc.ipynb), [1.1a_infer_masks](1.1a_infer_masks_from-composite_single_cell.ipynb), or [1.1b_infer_masks](1.1b_infer_masks_from-composite_multiple-cells.ipynb).***

### ➡️ **Input:**
In this workflow, the neuron cellmask segmentation will be used to "infer" the locations of the soma (or cell body) and the neurites as separate objects based on their shape. No intensity images are necessary for this workflow.

### 🍃 **Soma vs Neurites**
Neurons are morphologically complex cell types containing mulitple subcellular regions that each carry out specific functions. One basic subregion classification discernable in most confocal microscopy images of a neuron is the distinction between the soma (or cell body) and the neurites (e.g., dendrites and axons). Here, the segmentation of the neurites from the soma is included to enable downstream comparisons of organelles between subregions.

Organelle morphology, distribution, and interactions have been shown to be distinct between different functional regions of cells. For example, the morphological differences in mitochondria between the soma and neurites of a neuron show larger mitochondria volume in neurites than in the soma of the same cell [[1](https://pmc.ncbi.nlm.nih.gov/articles/PMC8423436/)]. Golgi have also been shown to differ in distribution throughout a neuron at different stages of neuronal development as the golgi engages in dendritic golgi translocation, eventually leaving golgi outposts during neuronal maturation [[2](https://doi.org/10.1016/j.celrep.2023.112709)]. Moreover, the interactions between ribosomes and the endoplasmic reticulum are more abundant in the soma and dendrites of neurons as compared to that of the axon [[3](https://pmc.ncbi.nlm.nih.gov/articles/PMC8143122/)].

Comparing across distinct subregions enables specific phenotypes underlying cellular physiology to be linked to morphologically distinct subregions of cells. The organelle morphology, interactions, and distribution measurements carried out in [Part 2](/notebooks/part_2_quantification/) are carried out and summarized for the entire cell and within each subregion as part of the Organelle Signature Analysis.


-----

### 👣 **Summary of steps**  

➡️ **EXTRACTION**
- **`STEP 1`** - Extraction of cellmask

    - Determine whether cellmask file is a multichannel image or not
    - If cellmask file is a multichannel image, choose the cellmask channel

**PRE-PROCESSING**
- **`STEP 2`** - Calculate radii of cell(s)
    
    - Choose between binary and isotropic segmentation methods
    - Rescale the cell mask to half its original size to reduce upcoming computational and memory requirements
    - Determine range of possible radii
    - Apply an erosion to rescaled mask using specified method & determine remaining volume
        - If no remaining volume, remove all radii above the used radii from radii range
        - If there is remaining volume, remove all radii below used radii from radii range 
        - Repeat until 1 or 2 values remain in the radii range
    - Assign determined radius to a label for its corresponding cell

**CORE PROCESSING**
- **`STEP 3`** - Segment initial soma mask

    - Determine radius for cell(s) based on their label
    - Apply an erosion the original cellmask using specified method
    - Dilate from post-erosion remaining values using the specified method
    - Re-mask image with original cellmask segmentation

- **`STEP 4`** - Segment initial neurite mask

    - Determine radius for cell(s) based on their label
    - Mask the inverse of the initial soma mask with the original cellmask
    - Remove neurite segmentation below size determined by radius

**POST-PROCESSING**
- **`STEP 5`** - Clean soma segmentation

    - Mask the inverse of the newly made neurite mask with the original cellmask to get small outcrops
    - Remove loose specks that are not connected to soma

- **`STEP 6`** - Clean neurite segmentation

    - Mask the inverse of the newly made soma mask with the original cellmask

**POST-POST-PROCESSING** 
- **`STEP 7`** - Stack masks

    - Stack masks in order of cleaned soma mask, cleaned neurites mask

**EXPORT** ➡️
- Save stacked masks to output file location


---------------------
## **IMPORTS AND LOAD IMAGE**
Details about the functions included in this subsection are outlined in the [`1.0_image_setup`](1.0_image_setup.ipynb) notebook. Please visit that notebook first if you are confused about any of the code included here.

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
from pathlib import Path
import os, sys
import numpy as np
from infer_subc.core.file_io import (list_image_files, 
                                     read_czi_image, 
                                     read_tiff_image,
                                     export_inferred_organelle)
from infer_subc.utils.batch import find_segmentation_tiff_files
from infer_subc.core.img import size_filter_linear_size, select_cellmask_from_img
from infer_subc.organelles.cellmask import (find_radius, 
                                            infer_soma_from_mask, 
                                            infer_neurites_from_mask, 
                                            clean_soma_from_neurites, 
                                            clean_neurites_from_soma)

import napari
from napari.utils.notebook_display import nbscreenshot
from skimage.morphology import isotropic_opening, isotropic_dilation, isotropic_erosion, binary_opening, binary_dilation, binary_erosion, disk, ball
from skimage.measure import label
from scipy.ndimage import zoom
import pandas as pd

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: `im_type`, `data_root_path`, `in_data_path`, and `out_data_path`.

In [ ]:
#### USER INPUT REQUIRED ###
im_type = ".tiff"
# data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"
# in_data_path = Path("C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/neurites/segmentations")
# out_data_path = Path("C:/Users/zscoman/Documents/Python Scripts/Infer-subc-2D/neurites/segmentations/out")

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D/neurites"
in_data_path = data_root_path / "raw"
out_data_path = data_root_path / "segmentations"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# list files in the input folder
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

#### &#x1F6D1; &#x270D; **User Input Required:**

Use the list above to specify which image you wish to analyze based on its index: `test_img_n`

In [ ]:
#### USER INPUT REQUIRED ###
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# load image and metadata
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# metadata
channel_names = meta_dict['name']
meta = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']
file_path = meta_dict['file_name']
print("Metadata information")
print(f"File path: {file_path}")
for i in list(range(len(channel_names))):
    print(f"Channel {i} name: {channel_names[i]}")
print(f"Scale (ZYX): {scale}")
print(f"Channel axis: {channel_axis}")

# open viewer and add images
viewer = napari.Viewer()
for i in list(range(len(channel_names))):
    viewer.add_image(img_data[i],
                     scale=scale,
                     name=f"Channel {i}")
    
cellmask = read_tiff_image(find_segmentation_tiff_files(file_path, ['cell'], out_data_path, '-')['cell'])
viewer.add_labels(cellmask, scale=scale, name="cellmask")
viewer.grid.enabled = True
viewer.reset_view()
print("\nProceed to Napari window to view your selected raw image.")

-----

## **EXTRACTION**

### **`STEP 1` - Read in Cell Mask**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please determine whether the cellmask file has both the nucleus and the cellmask within it.
- `nuc_present`: a True/False statement of whether the cellmask file contains the nucleus mask
If the cellmask does contain the nucleus mask, we will automatically collect only channel 0 (which should be the cellmask)
- `chan`: an int for the channel the cellmask is located in within the cellmask file for files with both the nucleus and cellmask stacked together. If nuc_present is set to False, this value can be ignored

In [ ]:
#### USER INPUT REQUIRED ###
multichannel_input = False
chan = 0

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block extracts the cell mask from the cell mask and nucleus mask combined image. It will be the only part of the original image used in the rest of this workflow. The cell mask channel is added to the Napari viewer.

In [ ]:
if multichannel_input:
    cell_mask = cellmask[chan]
    viewer.add_image(cell_mask, name="Cell Mask", scale=scale)
else:
    cell_mask = cellmask
    viewer.add_image(cell_mask, name="Cell Mask", scale=scale)
viewer.grid.enabled = False

-----
## **PRE-PROCESSING**

### **`STEP 2` - Calculate Radii of Cell(s)**

#### &#x1F6D1; &#x270D; **User Input Required:**

Decide whether to use isotropic or binary methods. Isotropic methods are more memory and time efficient; however, they are more likely to undersegment the soma as compared to binary methods. Binary methods are more likely to oversegment the soma as compared to the isotropic methods, so they should be used if having the initial segment of the neurites included in the soma region is desired.

The binary method uses a saucer-like shape as the footprint; whereas the isotropic method uses a spherical shape as the footprint. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space whereas the ball assumes the same number of voxels wide as it is tall and deep. 

In cases where the soma is more circular, using isotropic methods will be executed faster, and the more circular the cell is, less of the cell will be considered part of the neurites. The binary methods on the other hand will be more saucer shaped, allowing for more of a irregular shaped soma. Binary methods require higher processing power, and are executed slower than isotropic methods.

The determined radius is used to recreate a saucer or sphere for the actual segmentation of the soma from the neurites in the following step.

In [ ]:
# Options are 'isotropic' or 'binary'
method = 'binary'

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block contains a loop that continues until there are only 1 or 2 values remaining in the radii range for each cell within the image.

First, the block of code creates a range of possible radii. These radii are then sorted through by checking the efficacy of the radius in the middle of the range of possible radii, and removing either all radii above the tested radius value or below the tested radius value from the list. The above process is repeated until there are only 1 or 2 values remaining in the radii range.

The way that the code checks the efficacy of the radius in the range is either developing a version of the "saucer" that will be used to segment the soma and neurites from the cell mask or using an isotropic erosion method to segment the soma from the cellmask via a sphere. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space. The top and bottom disk are set to a shorter radius than the center disk. 

The efficacy of the radius is deemed as whether the radius results in an absence of cell mask or not. If there is an absence of cell mask, the radius is deemed too large, and it alongside all radii above it are removed from the possible radii. If there is no absence of cell mask, the radius is deemed too small, and all radii that are below it are removed from the possible radii. This process repeats until there are only 1 or 2 possible radii remaining, at which point, the average radius of the remaining radii is selected, and divided by 2.

When the final radius value is determined, the block of code outputs an image with the cell(s) labeled with the radius value and the cell number with no other modifications from the cell mask image. 
> NOTE: The label given to each cell is defined as $('radius$ $of$ $the$ $cell'$ $×$ $10^n)$ $+$ $'cell$ $number'$ where $n$ is the number of digits in total number of cells. 

> For example, in an image with 11 total cells, $'cell$ $number'$ 6 with a $radius$ of 16 would be given a label of 1606.

In [ ]:
radii_mask = np.zeros_like(cell_mask)
cell_mask_resize = zoom(cell_mask.copy(), (1, 0.5, 0.5))
zz, yy, xx = cell_mask_resize.shape

cell_nums = np.unique(cell_mask[cell_mask != 0])
label_factor = 10 ** cell_nums.max()

for cell_num in cell_nums:
    test_img = (cell_mask_resize == cell_num)
    rad_range = [i+1 for i in range(yy // 4)]  # Dividing by 4 because mask is resized

    if method == 'isotropic':
        while len(rad_range) > 2:
            rad = rad_range[len(rad_range) // 2]
            print(f"Trying radius of {rad}")
            if np.all(isotropic_erosion(test_img.astype(np.uint8), rad) == 0):
                rad_range = rad_range[:rad_range.index(rad)]
                print(f"{rad} is too large")
            else:
                rad_range = rad_range[rad_range.index(rad)+1:]
                print(f"{rad} is too small")
            print(f"{len(rad_range)} possible radii remaining")
    elif method == 'binary':
        while len(rad_range) > 2:
            rad = rad_range[len(rad_range) // 2]
            print(f"Trying radius of {rad}")
            edge = disk(rad // 4)
            middle = disk(rad)
            w = (middle.shape[0] - edge.shape[0]) // 2
            edge = np.pad(edge, ((w, w), (w, w)), mode='constant', constant_values=0)
            fp = np.stack((edge, middle, edge))
            if np.all(binary_erosion(test_img.astype(np.uint8), fp) == 0):
                rad_range = rad_range[:rad_range.index(rad)]
                print(f"{rad} is too large")
            else:
                rad_range = rad_range[rad_range.index(rad)+1:]
                print(f"{rad} is too small")
            print(f"{len(rad_range)} possible radii remaining")

    if len(rad_range) == 1:
        opti_rad = rad_range[0] // 2
    elif len(rad_range) == 2:
        opti_rad = (rad_range[0] + rad_range[1]) // 4
    radii_mask[cell_mask == cell_num] = (opti_rad * label_factor) + cell_num

In [ ]:
# radii_mask = np.zeros_like(cell_mask)
# cell_mask_resize = zoom(cell_mask.copy(), (1, 0.5, 0.5))
# zz, yy, xx = np.shape(cell_mask_resize)

# cell_nums = np.unique(cell_mask[cell_mask != 0])
# label_factor = 10 ** cell_nums.max()

# for cell_num in cell_nums:
#     rad_range = [i+1 for i in list(range(yy // 4))] #determines a range of radii to test for each cell
#                                                     # Dividing by 4 because the cell mask is resized to half the original size
#     test_img = np.zeros_like(cell_mask_resize)
#     test_img[cell_mask_resize == cell_num] = 1
#     if method == 'isotropic':
#         while len(rad_range) > 2:
#             rad = rad_range[((len(rad_range)) // 2)]
#             print(f"Trying radius of {rad}")
#             if np.array_equal(isotropic_erosion(test_img.copy(), rad), np.zeros_like(test_img)):
#                 rad_range = rad_range[:(rad_range.index(rad))]
#                 print(f"{rad} is too large")
#             else:
#                 rad_range = rad_range[(rad_range.index(rad)+1):]
#                 print(f"{rad} is too small")
#             print(f"{len(rad_range)} possible radii remaining")
#     elif method == 'binary':
#         while len(rad_range) > 2:
#             rad = rad_range[((len(rad_range)) // 2)]
#             print(f"Trying radius of {rad}")
#             edge = disk(rad//4)
#             middle = disk(rad)
#             w = (np.shape(middle)[0] - np.shape(edge)[0])//2
#             edge = np.pad(edge, ((w,w),(w,w)), mode = 'constant', constant_values=0)
#             fp = np.stack((edge, middle, edge))
#             if np.array_equal(binary_erosion(test_img, fp), np.zeros_like(test_img)):
#                 rad_range = rad_range[:(rad_range.index(rad))]
#                 print(f"{rad} is too large")
#             else:
#                 rad_range = rad_range[(rad_range.index(rad)+1):]
#                 print(f"{rad} is too small")
#             print(f"{len(rad_range)} possible radii remaining")

#     if len(rad_range) == 1:
#         opti_rad = rad_range[0]//2
#     elif len(rad_range) == 2:
#         opti_rad = (rad_range[0] + rad_range[1])//4
#     radii_mask[cell_mask == cell_num] = (opti_rad*label_factor) + cell_num

-----
## **CORE-PROCESSING**

### **`STEP 3` - Segment Initial Soma Mask**

#### &#x1F6D1; &#x270D; **User Input Required:**

For ideal functionality, you should use the same method here as was used above. Changing methods can result in parts of the soma being heavily under or oversegmented. In the worst case scenario, this may result in a complete absence of a soma; however, in the best case scenario, this may result in a soma that is segmented slightly better than what would have happened if you chose the same method as earlier.

In [ ]:
# Options are 'isotropic' or 'binary'
method = 'binary'

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This block of code output a soma segmentation for each cell using the radii determined in step 2.

To collect the radii determined in step 2, this block of code undoes the math used to label the cells in search of the cell's radius value. This radius value is then used to segment the soma from the cell mask in either the "saucer" used in binary segmentaiton or the sphere used in isotropic segmentation. The "saucer" is comprised of 3 2-Dimensional disks stacked into a 3-Dimensional space where the top and bottom disk are set to a shorter radius than the center disk. Meanwhile, the sphere is equidimensional in shape with the radius of it being exactly what was determined in step 2 at all angles.

While code in this step is similar to that of step 3, this step uses the enlarged version of the cell mask, not the resized version used for the determination of the optimal radius. This results in a more accurate version of the cell mask being segmented rather than the shrunken down version used to assist with processing speed and memory usage.

Afterwards, the soma segmentation is added to Napari for visualization.

In [ ]:
soma_out_1 = np.zeros_like(cell_mask)

cell_nums = np.unique(cell_mask[cell_mask != 0])
label_factor = 10 ** cell_nums.max()

for cell_num in cell_nums:
    soma_img_solo = (cell_mask == cell_num)
    opti_rad = np.unique(radii_mask[soma_img_solo])[0]
    opti_rad = (opti_rad - cell_num) / label_factor

    if method == 'isotropic':
        neurites_removed = isotropic_opening(soma_img_solo.astype(np.uint8), opti_rad)
        soma_initial = isotropic_dilation(neurites_removed, opti_rad) & soma_img_solo
    elif method == 'binary':
        edge = disk(int(opti_rad // 2))
        middle = disk(int(opti_rad))
        w = (middle.shape[0] - edge.shape[0]) // 2
        edge = np.pad(edge, ((w, w), (w, w)), mode='constant', constant_values=0)
        fp = np.stack((edge, middle, edge))
        neurites_removed = binary_opening(soma_img_solo.astype(np.uint8), fp)
        viewer.add_image(neurites_removed, scale=scale)
        soma_initial = binary_dilation(neurites_removed, footprint=ball(int(opti_rad // 2))) & soma_img_solo
        viewer.add_image(soma_initial, scale=scale)
    else:
        raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")

    soma_out_1[soma_initial] = cell_num

viewer.add_labels(soma_out_1, scale=scale, name="Initial Soma Segmentations")

In [ ]:
# soma_out_1 = np.zeros_like(cell_mask)

# cell_nums = np.unique(cell_mask[cell_mask != 0])
# label_factor = 10 ** cell_nums.max()

# for cell_num in cell_nums:
#     soma_img_solo = np.zeros_like(cell_mask)
#     soma_img_solo[cell_mask == cell_num] = 1

#     opti_rad = np.unique(radii_mask[cell_mask == cell_num])[0]
#     opti_rad = (opti_rad - cell_num) / label_factor

#     if method == 'isotropic':
#         neurites_removed = isotropic_opening(soma_img_solo, opti_rad)
#         soma_initial = isotropic_dilation(neurites_removed, (opti_rad)) * soma_img_solo
#     elif method == 'binary':
#         edge = disk(opti_rad//2)
#         middle = disk(opti_rad)
#         w = (np.shape(middle)[0] - np.shape(edge)[0])//2
#         edge = np.pad(edge, ((w,w),(w,w)), mode = 'constant', constant_values=0)
#         fp = np.stack((edge, middle, edge))
#         neurites_removed = binary_opening(soma_img_solo, fp)
#         viewer.add_image(neurites_removed, scale=scale)
#         soma_initial = binary_dilation(neurites_removed, footprint=ball((opti_rad)//2)) * soma_img_solo
#         viewer.add_image(soma_initial, scale=scale)
#     else: 
#         raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")

#     soma_out_1[soma_initial == 1] = cell_num #ensures that the only spot changed is the target cell
# viewer.add_labels(soma_out_1, scale=scale, name="Initial Soma Segmentations")

### **`STEP 4` - Segment Initial Neurite Mask**

#### &#x1F6D1; &#x270D; **User Input Required:**

Once again, for ideal functionality, you should use the same method here as was used above. Changing methods can result in parts of the neurites being excluded from the neurite segmentation that should otherwise be included or vice versa. In the worst case scenario, the parts of the neurites that should be included in the neurite segmentation will be instead included as a part of the soma later down the pipeline. In the best case scenario, there will be little to no difference from choosing the other method as this will be corrected when cleaning the soma and neurite segmentations later down the pipeline.

In [ ]:
# Options are 'isotropic' or 'binary'
method = 'binary'

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block uses the inverse of the soma found in the previous step to segment the neurites from the original cell mask.

Using the radii found in step 2, smaller sections of the neurites are temporarily removed from the segmentation to check if they would serve better as part of the soma. To collect the radii determined in step 2, this block of code also undoes the math used to label the cells in search of the cell's radius value. 

In [ ]:
neurites_out_1 = np.zeros_like(cell_mask)

cell_nums = np.unique(cell_mask[cell_mask != 0])
label_factor = 10 ** cell_nums.max()
binary_soma = soma_out_1 > 0

for cell_num in cell_nums:
    solo_mask = (cell_mask == cell_num)
    opti_rad = np.unique(radii_mask[solo_mask])[0]
    opti_rad = (opti_rad - cell_num) / label_factor

    neurite_mask = ~binary_soma & solo_mask
    if method == 'isotropic':
        filtered = size_filter_linear_size(img=label(neurite_mask), min_size=(opti_rad*2), method='3D') * solo_mask
    elif method == 'binary':
        filtered = size_filter_linear_size(img=label(neurite_mask), min_size=(opti_rad//2), method='3D') * solo_mask
    else:
        raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")

    neurite_labels = label(filtered)
    neurite_labels[neurite_labels > 0] = (neurite_labels[neurite_labels > 0] * label_factor) + cell_num
    neurites_out_1[solo_mask] = neurite_labels[solo_mask]

viewer.add_labels(neurites_out_1, scale=scale, name="Initial Neurite Segmentations")

In [ ]:
# neurites_out_1 = np.zeros_like(cell_mask)

# cell_nums = np.unique(cell_mask[cell_mask != 0])
# label_factor = 10 ** cell_nums.max()

# for cell_num in cell_nums:
#     solo_mask = np.zeros_like(cell_mask)
#     binary_soma = np.zeros_like(cell_mask)
#     solo_mask[cell_mask==cell_num] = 1
#     binary_soma[soma_out_1 > 0] = 1
#     opti_rad = np.unique(radii_mask[cell_mask == cell_num])[0]
#     opti_rad = (opti_rad - cell_num) / label_factor
#     if method == 'isotropic':
#         neurites1 = ~(binary_soma.astype(bool)) & (solo_mask == 1)
#         neurites1 = label(size_filter_linear_size(img=label(neurites1), min_size=(opti_rad*2), method='3D') * solo_mask)               #objects below a certain size are not considered neurites initially 
#     elif method == 'binary':
#         neurites1 = ~(binary_soma.astype(bool)) & (solo_mask == 1)
#         neurites1 = label(size_filter_linear_size(img=label(neurites1), min_size=(opti_rad//2), method='3D') * solo_mask)              #objects below a certain size are not considered neurites initially
#     else: 
#         raise ValueError(f"method of {method} was given, but only 'isotropic' or 'binary' is allowed.")
#     neurites1[neurites1 > 0] = (neurites1[neurites1 > 0] * label_factor) + cell_num # relabels the neurites
#     neurites_out_1[solo_mask == 1] = neurites1[solo_mask==1] #ensures that the only spot changed is the target cell

# viewer.add_labels(neurites_out_1, scale=scale, name="Initial Neurite Segmentations")

-----
## **POST-PROCESSING**

### **`STEP 5` - Cleaned Soma Segmentation**

#### &#x1F3C3; **Run code; no user input required**

The below block of code takes smaller areas of the neurite segmentation, and checks if they are directly connected to the soma segmentation. If so, these areas are considered a part of the soma rather than the neurites. If there are parts of the neurite mask that were not directly connected with the soma mask, those parts are temporarily removed to be included in the neurite mask in the following step.

In [ ]:
soma_out_2 = np.zeros_like(cell_mask)

cell_nums = np.unique(cell_mask[cell_mask != 0])
label_factor = 10 ** cell_nums.max()

# Create a mask for all neurites at once
neurites_mask = (neurites_out_1 % label_factor) > 0

# For each cell, mask soma regions in one go
soma_mask = (~neurites_mask) & (cell_mask != 0)

# Find the most common value in soma_mask for each cell and assign only those pixels
for cell_num in cell_nums:
    cell_region = (cell_mask == cell_num)
    soma_region = soma_mask & cell_region
    # Only keep the largest connected region (most common value)
    if np.any(soma_region):
        bincount = np.bincount(soma_region.ravel())
        main_val = np.argmax(bincount[1:]) + 1 if len(bincount) > 1 else 1
        soma_region = soma_region & (soma_region == main_val)
        soma_out_2[cell_region] = soma_region[cell_region] * cell_num

viewer.add_labels(soma_out_2, scale=scale, name="Cleaned Soma Segmentation")

In [ ]:
# soma_out_2 = np.zeros_like(cell_mask)

# cell_nums = np.unique(cell_mask[cell_mask != 0])
# label_factor = 10 ** cell_nums.max()

# for cell_num in cell_nums:
#     solo_mask = np.zeros_like(cell_mask)
#     solo_mask[cell_mask==cell_num] = 1
#     neurites = np.zeros_like(cell_mask)
#     neurites[(neurites_out_1 % label_factor) == cell_num] = 1
#     soma = ~(neurites.astype(bool)) & (solo_mask == 1)
#     soma[soma!=np.bincount(np.ravel(soma)[np.ravel(soma)!= 0]).argmax()] = 0
#     soma_out_2[cell_mask == cell_num] = soma[cell_mask == cell_num] * cell_num
    
# viewer.add_labels(soma_out_2, scale=scale, name="Cleaned Soma Segmentation")

### **`STEP 6` - Cleaned Neurite Segmentation**

#### &#x1F3C3; **Run code; no user input required**

The below block of code uses the cleaned up soma segmentation to resegment the neurites from the cell mask. Any sections of the neurites that were small, but were not directly attached to the soma are reconsidered as neurites whereas any small sections of the neurites that were directly attached to the soma are no longer considered neurites. This section of code also ensures that any section of the cell mask that the neurites are present in, the soma is not and vice versa.

In [ ]:
cell_nums = np.unique(cell_mask[cell_mask != 0])
binary_soma = soma_out_2 > 0

# Create a mask for all neurites at once
neurites_mask = (~binary_soma) & (cell_mask != 0)

# Label all neurite regions in one call
neurites_labels = label(neurites_mask)

# Relabel to encode cell number
label_factor = 10 ** cell_nums.max()
neurites_out_2 = np.zeros_like(cell_mask)
for cell_num in cell_nums:
    neurites_out_2[(cell_mask == cell_num) & (neurites_mask > 0)] = (neurites_labels[(cell_mask == cell_num) & (neurites_mask > 0)] * label_factor) + cell_num

viewer.add_labels(neurites_out_2, scale=scale, name="Cleaned Neurite Segmentations")

In [ ]:
# neurites_out_2 = np.zeros_like(cell_mask)

# cell_nums = np.unique(cell_mask[cell_mask != 0])
# label_factor = 10 ** cell_nums.max()

# for cell_num in cell_nums:
#     solo_mask = np.zeros_like(cell_mask)
#     solo_mask[cell_mask==cell_num] = 1
#     binary_soma = np.zeros_like(cell_mask)
#     binary_soma[soma_out_2 > 0] = 1
#     neurites = ~(binary_soma.astype(bool)) & (solo_mask == 1)
#     neurites[neurites > 0] = (label(neurites[neurites > 0]) * label_factor) + cell_num
#     neurites_out_2[solo_mask == 1] = neurites[solo_mask == 1] #ensures that the only spot changed is the target cell

# neurites_out_2 = label(neurites_out_2)
# viewer.add_labels(neurites_out_2, scale=scale, name="Cleaned Neurite Segmentations")

-----
## **POST-POST-PROCESSING**

### **`STEP 7` - Stacking Soma & Neurites**

#### &#x1F3C3; **Run code; no user input required**

The below block of code stacks the masks into one file for exports. These masks will need to be unstacked through the [quality_check_segmentations notebook](quality_check_segmentations.ipynb) to be used in the quantification workflows.

In [ ]:
soma_neurites = np.stack([soma_out_2, neurites_out_2])
viewer.add_labels(soma_neurites, scale=scale, name="Soma and Neurites Stacked")

-----
## **SAVING**

## **`Saving` - Save the segmentation output**

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block saves the instance segmentation output to the `out_data_path` specified earlier.

In [ ]:
# Saving file
out_file_n = export_inferred_organelle(soma_neurites, "soma_neurites", meta_dict, out_data_path)
print(f"saved to: {out_data_path}")

-----
-----
## **Define `infer_soma_neurites()` function**
The following code includes an example of how the workflow steps above are combined into one function. This function can be run below to process a single image. It is included in the [batch process notebook](batch_process_segmentations.ipynb) to run the above segmentation on multiple images. 

#### &#x1F3C3; **Run code; no user input required**

### Define Function to Segment Soma & Neurites
infer_soma_neurites() takes in a mask segmentation and outputs a stacked numpy array containing the soma and the neurites in different channels.

In [ ]:
def _infer_soma_neurites(in_seg: np.ndarray, multichannel_input: bool=False, chan: int=0, method: str='binary'):

    ###################
    # EXTRACT
    ###################  
    cell_mask = select_cellmask_from_img(in_seg, multichannel_input=multichannel_input, chan=chan)

    ###################
    # PRE_PROCESSING
    ################### 
    radii_mask = find_radius(cell_mask, method)

    ###################
    # CORE_PROCESSING
    ###################
    soma_initial = infer_soma_from_mask(cell_mask, radii_mask, method)

    neurites_initial = infer_neurites_from_mask(cell_mask, radii_mask, soma_initial, method)

    ###################
    # POST_PROCESSING
    ################### 
    soma_cleaned = clean_soma_from_neurites(cell_mask, neurites_initial)

    neurites_cleaned = clean_neurites_from_soma(cell_mask, soma_cleaned)

    ###################
    # POST_POST_PROCESSING
    ################### 
    soma_neurites = np.stack([soma_cleaned, neurites_cleaned])
    
    return soma_neurites

In [ ]:
somneu = _infer_soma_neurites(cell_mask,  multichannel_input=multichannel_input, chan=chan, method=method)

#confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(somneu == soma_neurites)}")

In [ ]:
# import soma_neurite function from infer_subc.organelles.cellmask and run soma and neurite segmentation function 
from infer_subc.organelles.cellmask import infer_soma_neurites
som_neu_obj = infer_soma_neurites(cell_mask,  multichannel_input=multichannel_input, chan=chan, method=method)


#confirm this output matches the output saved above
print(f"The segmentation output here matches the output created above: {np.all(soma_neurites == som_neu_obj)}")

# adding image to Napari as a new layer
viewer.add_labels(somneu, scale=scale, name="infer_soma_neurites() output")
viewer.grid.enabled = True
viewer.reset_view()

# screenshot viewer
nbscreenshot(viewer, canvas_only = False)


-------------
### ✅ **INFER LIPID DROPLETS COMPLETE!**

Continue on to other notebooks as needed:
- Infer [`lysosomes`](1.2_infer_lysosome.ipynb)
- Infer [`mitochondria`](1.3_infer_mitochondria.ipynb)
- Infer [`golgi`](1.4_infer_golgi.ipynb)
- Infer [`peroxisomes`](1.5_infer_peroxisome.ipynb)
- Infer [`endoplasmic reticulum (ER)`](1.6_infer_ER.ipynb)
- Infer [`lipid droplets`](1.7_infer_lipid_droplet.ipynb)

Or proceed to batch processing here: [batch process notebook](batch_process_segmentations.ipynb)